<a href="https://colab.research.google.com/github/isegura/OCW-UC3M-NLPDeep-2023/blob/main/tema5_2_dataset_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# 5.2. Cómo cargar un dataset en un objeto Dataset (Hugging Face)
En el ejercicio anterior aprendimos a cargar un dataset alojado en Hugging Face (https://huggingface.co/datasets) usando la librería **datasets**.

En este ejercicio, veremos cómo es posible además cargar un dataset almacenado localmente o en un servidor y guardarlo en un objeto **Dataset** de la librería **datasets**. Es cierto, que podríamos seguir cargandolo en un dataframe de Pandas, pero para trabajar con transformers, utilizar las clases **DatasetDict** y **Dataset** de la librería **datasets** nos faciliará mucho el trabajo a la hora de procesar los datos y pasarselos al modelo.

Además, también aprenderemos a trabajar con algunos métodos útiles para trabajar con dataset, tales como **filter**, **map**, **sort**, **select** y **shuffle**.


En este ejercicio, vamos a utlizar un dataset proporcionado por Kaggle para la tarea de detección de sacarmo. Puedes encontrarlo en el siguiente enlace:

https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection

El dataset está formado por títulos de noticias de periódicos online, y cada uno de estos títulos ha sido clasificado con 1, si el título usa el sarcasmo, y 0, si el título no usa el sarcasmo.


Descarga los tres ficheros del dataset y guardalos en tu carpeta 'Colab Notebooks/data/sarcasm/' de tu unidad de google drive.


**NOTA PARA PODER EJECUTAR ESTE NOTEBOOK**:

1) Para poder ejercutar correctamente este notebook, deberás abrirlo en tu Google Drive (por ejemplo, en la carpeta 'Colab Notebooks').

2) Además, debes guardar el dataset en tu Google Drive, dentro de carpeta 'Colab Notebooks/data/sarcasm/'.

## Montar unidad de google drive y cambiar directorio de trabajo

Como ya hemos hecho en otros ejercicioss, lo primero que tenemos que hacer es montar nuestra unidad de google drive y modificar el directorio de trabajo actual para que sea el directorio donde está almacenado el dataset:



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/data/sarcasm/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


En la siguiente celda, comprobamos que en efecto la carpeta contiene los tres ficheros que hemos descargado de Kaggle:

In [ ]:
!ls

test.csv  train.csv  val.csv


## Instalación de la librería datasets

In [ ]:
!pip install -q datasets

## Cargar el dataset desde local

Ahora ya sí estamos preparados para cargar el dataset. Para ello simplemente, como primer argumento de la función **load_dataset**, debemos pasar la cadena **'csv'**. De esta forma, estamos indicando que vamos a cargar el dataset desde local y en formato csv. Como segundo argumento, vamos a pasarle un diccionario **data_files**, cuyas claves son los nombres de los splits que vamos a crear (en nuestro caso son tres: train, validation y test), y asociado a cada clave, la ruta al fichero que contiene las instancias de dicho split. Como en la primera celda ya modificamos el directorio de trabajo actual a la carpeta que almacena los tres ficheros, bastará con indicar el nombre de cada fichero (es decir, su ruta relativa).

In [ ]:
from datasets import load_dataset

data_files = {"train": "train.csv",
              "validation": "val.csv",
              "test": "test.csv"}

dict_sarcasm= load_dataset("csv", data_files=data_files)
dict_sarcasm

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'headline', 'is_sarcastic'],
        num_rows: 19952
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'headline', 'is_sarcastic'],
        num_rows: 2850
    })
    test: Dataset({
        features: ['Unnamed: 0', 'headline', 'is_sarcastic'],
        num_rows: 5701
    })
})

Como resultado podemos ver que se ha creado un objeto **DatasetDict** que es un diccionario con tres claves, una para cada split: train, validation y test.
Podemos ver que el número de filas (**num_rows**), que son el número de instancias de cada split, es distinto para cada uno de ellos, siendo el más grande el de train (con 19.952 instancias) y el más pequeño el de validación, sólo con 2.850.

Además, también podemos ver que cada instancia está formada por tres campos (features):
- 'Unnamed: 0': es una especie de identificador, pero es un campo que realmente no va a ser utilizado.
- 'headline': texto del titular
- 'is_sarcastic': 0, si el titular no es sarcástico, 1 en otro caso.



## Cargar dataset desde un remoto
También podemos cargar un dataset alojado un servidor. Otra vez deberemos indicar el formato del dataset (consulta el siguiente enlace para ver qué formatos están permitidos https://huggingface.co/docs/datasets/loading) , en el caso del ejemplo **json**, y un diccionario con los ficheros que componen el dataset, indicando para cada uno el nombre del split y su url.

En la celda siguiente vamos a cargar el dataset Stanford Question Answering Dataset (SQUAD) para la tarea de question answering. El dataset se compone de pares de textos, formado por una pregunta y su respuesta.


In [ ]:
url = "https://github.com/crux82/squad-it/raw/master/"
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}
load_dataset("json", data_files=data_files, field="data")


DatasetDict({
    train: Dataset({
        features: ['paragraphs', 'title'],
        num_rows: 442
    })
    test: Dataset({
        features: ['paragraphs', 'title'],
        num_rows: 48
    })
})

## Cómo acceder a la información

Recuerda que para acceder a los datos de una instancia, tendrás que indicar primero su split, el índice de la instancia, y luego el campo que quieras consultar. Vamos a ver algunos ejemplos sobre el dataset para la tarea de detección de sarcasmo que cargamos al principio:


In [ ]:
print("Primera instancia del training:", dict_sarcasm['train'][0])
# Primera instancia del training: {'Unnamed: 0': 23789, 'headline': 'the great vanishing', 'is_sarcastic': 0}

print("Última instancia de test:", dict_sarcasm['test'][-1])
# Última instancia de test: {'Unnamed: 0': 22846, 'headline': 'supreme court justice sotomayor continues duties after breaking shoulder', 'is_sarcastic': 0}


Primera instancia del training: {'Unnamed: 0': 23789, 'headline': 'the great vanishing', 'is_sarcastic': 0}
Última instancia de test: {'Unnamed: 0': 22846, 'headline': 'supreme court justice sotomayor continues duties after breaking shoulder', 'is_sarcastic': 0}



Las siguientes dos instrucciones devolverán el mismo resultado (el título de la primera instancia del split train):

```
1) dict_sarcasm['train']['headline'][0]
2) dict_sarcasm['train'][0]['headline']

```


¿son igual de eficientes?, en caso contrario, ¿cuál es la opción más eficiente?

En la siguiente celda, puedes incluir código para medir el tiempo que tarda el acceso a cada una de las llamadas 1) y 2). Una opción sencilla es usar el paquete **time** y su función **time()**:


En esta primera opción, para consultar el primer título, accedemos primero al campo headline del train, y luego a la instancia con índice 0:

In [ ]:
import time

start = time.time()
dict_sarcasm["train"]["headline"][0]
end = time.time()

time_required_feature_index = end - start

print('Obtener dict_sarcasm["train"]["text"][0] tardó: ', time_required_feature_index, 'segundos')


Obtener dict_sarcasm["train"]["text"][0] tardó:  0.03412818908691406 segundos


En esta segunda opción, para acceder al primer título, accedemos a la primera de las instancias del split train, y luego a su campo headline

In [ ]:

start = time.time()
dict_sarcasm['train'][0]["headline"]
end = time.time()
time_required_index_feature = end - start

print('Obtener dict_sarcasm["train"][0]["text"] tardó: ', time_required_index_feature, 'segundos')


Obtener dict_sarcasm["train"][0]["text"] tardó:  0.0016179084777832031 segundos


¿Por qué la segunda opción es más eficiente? Es más eficiente porque únicamente necesita cargar el primer registro y acceder a su campo 'headline'. Sin embargo, en la primera opción, es necesario recuperar toda la lista de 'headlines', y luego devolver su primer valor.

**Por tanto, el orden de los campos a la hora de acceder a la información sí es importante.**
